In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
from os import listdir
from os.path import isfile, join
import urllib.request
import numpy as np
import random
from shutil import copyfile
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

# 데이터 증강 (Image Augmentation)

### ※※※※※ 한 번 돌렸으니 절대 다시 돌리지 말것!!!!!!!!! 오래 걸림!!! ※※※※※

In [ ]:
'''
data_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15, 
                                   shear_range=0.5,  
                                   # width_shift_range=0.1,
                                   # height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest') 
 
pre_dir = '/content/gdrive/MyDrive/클플test/data/data/train'
folders = os.listdir(pre_dir) # 해당 폴더에 있는 파일 이름을 리스트 형태로 받음
folders.sort() 

filename_in_dir = [] 

for i in range(len(folders)):
  path = pre_dir + "/" + folders[i] 
  os.chdir(path) # 해당 폴더로 이동
  files = os.listdir(path) # 해당 폴더에 있는 파일 이름을 리스트 형태로 받음
  for j in range(len(files)):
    pre_image = path + '/' + files[j]
    img = load_img(pre_image) 
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    k = 0
    for batch in data_datagen.flow(x,save_to_dir=pre_dir+'/'+folders[i], save_prefix='augmentation', save_format='jpg'):
        k += 1
        if k > 10: # 최대 6장씩 부풀리기
            break
'''

# trian, validation 이미지 데이터 조정

In [14]:
train_dir = '/content/gdrive/MyDrive/클플test/data/data/train'
validation_dir = '/content/gdrive/MyDrive/클플test/data/data/validation'
test_dir = '/content/gdrive/MyDrive/클플test/data/data/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=50, # step 1회 당 사용하는 데이터 수
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

Found 37538 images belonging to 33 classes.
Found 496 images belonging to 33 classes.


In [15]:
# 이미지 데이터 사이즈 / 레이블 크기

for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (50, 150, 150, 3)
배치 레이블 크기: (50, 33)


--------------------

# 전이학습 모델 Inception V3

- 전이학습 코드 참고 : https://codetorial.net/tensorflow/transfer_learning.html

In [16]:
# pre-trained 가중치 다운로드 -> tmp 폴더에 저장
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
  -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-06-11 05:57:15--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 74.125.137.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   263MB/s    in 0.3s    

2021-06-11 05:57:15 (263 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [17]:
# pre-trained 가중치 불러오기

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                              include_top=False,
                              weights=None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, 74, 74, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, 74, 74, 32)   96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 74, 74, 32)   0           batch_normalization_376[0][0]    
_______________________________________________________________________________________

In [18]:
# 마지막층 출력 확인
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output.shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


- mixed7이라는 층을 불러와서 output layer로 사용하는 건데, Inception V3를 다중분류에 활용한 (카테고리 10개) 코드 참고는 https://wikidocs.net/51543
- 예측까지 있음

## 모델 구성 / 컴파일

In [21]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## 모델 학습

- ※※※ 아주 오래 걸림 ※※※ 
- 그래서 일단 epoch 2로 학습

history = model.fit(
  train_generator,
  validation_data=validation_generator,
  steps_per_epoch=100,
  epochs=20,
  validation_steps=50,
  verbose=1)

In [25]:
history = model.fit(
            train_generator,
            validation_data=validation_generator,
            steps_per_epoch=100,
            epochs=2,
            verbose=1)

Epoch 1/2
100/100 [==============================] - 1828s 18s/step - loss: 1.1101 - accuracy: 0.9818 - val_loss: 1.1003 - val_accuracy: 0.9833
Epoch 2/2
100/100 [==============================] - 1384s 14s/step - loss: 0.6108 - accuracy: 0.9898 - val_loss: 0.8604 - val_accuracy: 0.9855


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')  ### acc 아니고 accuracy 일 수도
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

- test image로 예측하기는 https://airsbigdata.tistory.com/220 참고하기

# test 예측

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # image 예측하기
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a cat")
  else:
    print(fn + " is a dog")